# Import libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from collections import Counter

# Step 1: Load datasets

In [2]:

# Step 1: Load datasets
file_path = 'dataset.xlsx'  # Ubah ke lokasi file Anda
data_ik = pd.read_excel(file_path, sheet_name='DATA IK')
data_ib = pd.read_excel(file_path, sheet_name='DATA IB')

# Step 2: Preprocessing - Combine and clean data

In [3]:
# Step 2: Preprocessing - Combine and clean data
data_ik = data_ik[['tujuan']].rename(columns={'tujuan': 'keperluan'})
data_ib = data_ib[['desc']].rename(columns={'desc': 'keperluan'})
data_combined = pd.concat([data_ik, data_ib], ignore_index=True)
data_combined.dropna(subset=['keperluan'], inplace=True)

# Step 3: Clean text data
Regex (Regular Expression) adalah alat yang sangat berguna untuk memanipulasi teks. Dengan regex, Anda dapat mendefinisikan pola yang kompleks untuk menemukan dan mengubah bagian teks tertentu dengan mudah.

In [4]:
# Step 3: Clean text data
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Menghapus simbol dan angka
    text = re.sub(r"\s+", " ", text).strip()  # Menghapus whitespace berlebih
    return text

data_combined['keperluan'] = data_combined['keperluan'].apply(clean_text)


In [5]:
# Step 4: Count unique words
all_text = " ".join(data_combined['keperluan'])
word_counts = Counter(all_text.split())
top_words = word_counts.most_common(10)  # TOP 10 words
print("Top 10 Words by Frequency:")
for word, count in top_words:
    print(f"{word}: {count}")

Top 10 Words by Frequency:
pulang: 25264
rumah: 19718
ke: 18486
bertemu: 15078
libur: 14995
orang: 11005
tua: 10881
keluarga: 10727
kerumah: 9388
dan: 7383


In [6]:
# Step 5: Categorize "keperluan"
def classify_keperluan(text):
    if re.search(r"(laptop|printer|service|barang)", text):
        return "Memperbaiki Barang"
    elif re.search(r"(orangtua|keluarga|pulang|rumah|bertemu)", text):
        return "Bertemu Keluarga"
    elif re.search(r"(proyek|PKM|tugas|makalah|observasi|akhir|data)", text):
        return "Tugas Kuliah"
    elif re.search(r"(kontrol|gigi|sakit|berobat|obat)", text):
        return "Pergi ke Klinik"
    elif re.search(r"(membeli|pribadi|mengurus|rekening|tabungan)", text):
        return "Urusan Pribadi"
    elif re.search(r"(wisuda|graduation|clearance)", text):
        return "Tamat Kuliah"
    else:
        return "Lainnya"

data_combined['kategori'] = data_combined['keperluan'].apply(classify_keperluan)

In [7]:

# Mengurutkan kategori berdasarkan jumlah
kategori_counts = data_combined['kategori'].value_counts()
data_combined['kategori'] = pd.Categorical(data_combined['kategori'], categories=kategori_counts.index, ordered=True)

# Step 6: TF-IDF Vectorization

- Reprensentasi Fitur yang Efisien: TF-IDF (Term Frequency-Inverse Document Frequency) mengukur pentingnya kata dalam sebuah dokumen relatif terhadap seluruh korpus. Ini memberikan bobot lebih tinggi pada kata-kata yang jarang tetapi informatif.
- Mengurangi Dimensi: Dengan mengatur max_features=500, hanya 500 fitur yang paling relevan yang dipertahankan, sehingga mengurangi dimensi data dan mengurangi risiko overfitting.
- Stop Words Removal: Menggunakan daftar stop words dalam Bahasa Indonesia dari Sastrawi untuk menghapus kata-kata umum yang tidak membawa informasi penting, seperti "dan", "atau", "yang", dll.
#### TF-IDF merupakan metode yang sangat populer dan efektif dalam pencarian informasi dan analisis teks karena bisa memberikan representasi numerik yang lebih baik dari teks tanpa kehilangan konteks penting. Ini juga memungkinkan model untuk membedakan antara kata-kata yang sering muncul di hampir semua dokumen (yang akan memiliki bobot rendah) dan kata-kata yang jarang tetapi penting.

In [8]:
# Step 6: TF-IDF Vectorization
factory = StopWordRemoverFactory()
stop_words_indonesian = factory.get_stop_words()

vectorizer = TfidfVectorizer(max_features=500, stop_words=stop_words_indonesian)
X = vectorizer.fit_transform(data_combined['keperluan']).toarray()
y = data_combined['kategori']


# Step 7: Train-Test Split

#### Evaluasi Model yang Objektif: Pembagian data menjadi dua set (train dan test) memungkinkan untuk mengevaluasi model secara adil. Stratify=y memastikan bahwa proporsi kategori target di data latih dan uji tetap seimbang, yang penting untuk menghindari bias.
#### Generalization: Dengan membagi data, model dapat dilatih pada data yang berbeda dari data yang digunakan untuk evaluasi, yang membantu memastikan model tidak overfit dan dapat bekerja dengan baik pada data baru.

X_train: Fitur untuk data latih, yang akan digunakan untuk melatih model.

X_test: Fitur untuk data uji, yang akan digunakan untuk menguji kinerja model setelah dilatih.

y_train: Label untuk data latih, yang digunakan untuk melatih model.

y_test: Label untuk data uji, yang digunakan untuk mengevaluasi kinerja model

In [9]:
# Step 7: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 8: Hyperparameter Tuning for Decision Tree

#### Efisiensi dalam Penyetelan Parameter: GridSearchCV mengotomatisasi pencarian untuk parameter terbaik, yang akan memaksimalkan kinerja model tanpa memerlukan eksperimen manual.

 - Pencarian Parameter Terbaik: GridSearchCV memungkinkan kita untuk mencari secara sistematis kombinasi parameter yang memberikan hasil terbaik, yang dapat meningkatkan kinerja model.
 - Cross-Validation: Dengan menggunakan cv=5, Anda memastikan bahwa model diuji pada beberapa subset data, yang memberikan hasil yang lebih stabil dan mengurangi kemungkinan overfitting pada data latih tertentu.

# Decision Tree Classifier
#### Interpretabilitas dan Sederhana: Decision Tree mudah dipahami, bahkan oleh orang yang tidak berpengalaman dalam machine learning. Ini cocok untuk aplikasi yang memerlukan transparansi tentang bagaimana keputusan dibuat.
#### Kemampuan untuk menangani berbagai jenis data: Decision Tree bisa menangani data kategorikal maupun numerik dengan baik, yang membuatnya sangat fleksibel.
#### Tahan Terhadap Overfitting (dengan tuning): Dengan pengaturan hyperparameter yang tepat (seperti kedalaman maksimum), Decision Tree dapat memberikan performa yang sangat baik tanpa overfitting.

#### - Interpretabilitas dan Sederhana: Decision Tree mudah dipahami, bahkan oleh orang yang tidak berpengalaman dalam machine learning. Ini cocok untuk aplikasi yang memerlukan transparansi tentang bagaimana keputusan dibuat.
#### - Kemampuan untuk menangani berbagai jenis data: Decision Tree bisa menangani data kategorikal maupun numerik dengan baik, yang membuatnya sangat fleksibel.
#### - Tahan Terhadap Overfitting (dengan tuning): Dengan pengaturan hyperparameter yang tepat (seperti kedalaman maksimum), Decision Tree dapat memberikan performa yang sangat baik tanpa overfitting.

In [ ]:
# Step 8: Hyperparameter Tuning for Decision Tree
dt_model = DecisionTreeClassifier(random_state=42)
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [10, 20, 30],
    'min_samples_leaf': [5, 10, 15]
}
grid_search = GridSearchCV(dt_model, param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Parameters for Decision Tree:", best_params)

# Train the Decision Tree with best parameters
dt_model = DecisionTreeClassifier(random_state=42, **best_params)
dt_model.fit(X_train, y_train)


In [ ]:

# Step 9: Evaluate Model
y_pred = dt_model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('Actual', fontsize=12)
plt.show()


Decision tree memungkinkan kita untuk melihat bagaimana model membuat keputusan berdasarkan fitur. Ini meningkatkan kepercayaan terhadap model, karena kita bisa memahami jalur yang diambil untuk sampai pada keputusan tertentu.

In [ ]:
# 1. Import library yang dibutuhkan
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

# 2. Buat model
dt_model = DecisionTreeClassifier(
    max_depth=5,
    min_samples_split=50,
    min_samples_leaf=20,
    random_state=42
)

# 3. Training model dengan data
dt_model.fit(X_train, y_train)  # Pastikan X_train dan y_train sudah didefinisikan

# 4. Visualisasi tree
plt.figure(figsize=(10, 6))
plot_tree(dt_model, class_names=dt_model.classes_, filled=True, fontsize=6)
plt.title("Decision Tree Visualization")
plt.show()

Keuntungan Penggunaan:

Unsupervised Learning: K-Means adalah algoritma unsupervised learning yang mengelompokkan data berdasarkan kesamaan fitur. Ini berguna ketika Anda tidak memiliki label kategori untuk semua data, tetapi ingin menemukan pola atau struktur dalam data.
Menemukan Pola dalam Data: Dengan menggunakan PCA untuk mereduksi dimensi, Anda dapat visualisasikan cluster dalam dua dimensi dan memahami bagaimana data dikelompokkan. Ini membantu dalam pencarian pola dan pemahaman lebih dalam mengenai data.
Kenapa Memilih K-Means:

Simplicity dan Efektivitas: K-Means adalah salah satu algoritma clustering yang paling sederhana dan efektif, cocok untuk menemukan pola dalam dataset yang besar tanpa membutuhkan label.

In [ ]:
# Step 11: K-Means Clustering
pca = PCA(n_components=2)  # Reduksi ke 2 dimensi
X_pca = pca.fit_transform(X)
kmeans = KMeans(n_clusters=5, random_state=42, n_init=20).fit(X_pca)
y_kmeans = kmeans.predict(X_pca)

plt.figure(figsize=(10, 8))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_kmeans, cmap='viridis', marker='o', s=30, label='Data Points')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c='red', marker='x', s=200, label='Centroids')
plt.title("KMeans Clustering Visualization (2D)")
plt.xlabel("PCA Feature 1")
plt.ylabel("PCA Feature 2")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Step 12: Add Business Opportunities Based on Category
recommendations = {
    "Bertemu Keluarga": "Jasa Transportasi dan Ruang Pertemuan",
    "Pergi ke Klinik": "Klinik Kesehatan atau Apotek",
    "Tugas Kuliah": "Pusat Fotokopi atau Jasa Pengetikan",
    "Memperbaiki Barang": "Toko Elektronik atau Reparasi Barang",
    "Urusan Pribadi": "Layanan Logistik atau Pengurusan Dokumen",
    "Tamat Kuliah": "Jasa Fotografi atau Penyewaan Kostum",
    "Lainnya": "Usaha General seperti Minimarket"
}


In [ ]:

# Step 13: Create Summary Table for Business Opportunities
summary_table = pd.DataFrame({
    "Kategori": kategori_counts.index,
    "Peluang Bisnis": [recommendations.get(kategori, "Lainnya") for kategori in kategori_counts.index]
})

print("\nSummary Table with Business Opportunities:")
print(summary_table)

In [ ]:
# Step 14: Export categorized data (optional)
data_combined.to_csv("categorized_keperluan.csv", index=False)
print("Data dengan kategori berhasil diekspor ke 'categorized_keperluan.csv'.")